In [26]:
import tensorflow as tf
from keras.layers import Conv2D, MaxPooling2D, Activation, Dropout, Flatten, Dense, GlobalAveragePooling2D
from keras.applications.inception_v3 import InceptionV3
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Model,Sequential
from keras.optimizers import RMSprop, Adam
from keras import backend as K

In [4]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 12435211327219962545
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 7017170208
locality {
  bus_id: 1
  links {
  }
}
incarnation: 9987303805570526535
physical_device_desc: "device: 0, name: NVIDIA GeForce GTX 1080, pci bus id: 0000:01:00.0, compute capability: 6.1"
]


In [27]:
height = 299
width = 299
batch_size = 4

train_datagen = ImageDataGenerator(rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=False,
    validation_split=0.2)

train_generator = train_datagen.flow_from_directory(
    'C:/Users/stolk/Desktop/Workspaces/ML/malaria/high_res/data',#'/content/drive/My Drive/malariaNN/high_res/data',
    target_size=(height, width),
    batch_size=batch_size,
    class_mode='categorical',
    subset='training')
validation_generator = train_datagen.flow_from_directory(
    'C:/Users/stolk/Desktop/Workspaces/ML/malaria/high_res/data',#'/content/drive/My Drive/malariaNN/high_res/data',
    target_size=(height, width),
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation')

epochs = 20
steps = 170
val_steps = 40

Found 170 images belonging to 4 classes.
Found 40 images belonging to 4 classes.


In [28]:
base_model = InceptionV3(weights='imagenet', include_top=False)

out = base_model.output
out = GlobalAveragePooling2D()(out)
# fully-connected layer
out = Dense(1024, activation='relu')(out)
predictions = Dense(4, activation='softmax')(out)

model = Model(inputs=base_model.input, outputs=predictions)

# freeze all weights in imagenet trained base model, but not the extra buffer layer and output layer
for layer in base_model.layers:
    layer.trainable = False

    
    
def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))
    

LR = 0.007
RMS = Adam(lr=LR)
model.compile(optimizer=RMS, loss='categorical_crossentropy', metrics=['accuracy',f1_m,precision_m, recall_m])

In [29]:
history = model.fit(
    train_generator,
    epochs=epochs,
    steps_per_epoch=steps // batch_size,
    validation_data=validation_generator,
    validation_steps=val_steps // batch_size
)

Epoch 1/20
42/42 [==============================] - 19s 341ms/step - loss: 9.8897 - accuracy: 0.4984 - f1_m: 0.4404 - precision_m: 0.4479 - recall_m: 0.4347 - val_loss: 1.5026 - val_accuracy: 0.6500 - val_f1_m: 0.6500 - val_precision_m: 0.6500 - val_recall_m: 0.6500
Epoch 2/20
42/42 [==============================] - 13s 303ms/step - loss: 1.1446 - accuracy: 0.6942 - f1_m: 0.6938 - precision_m: 0.6942 - recall_m: 0.6935 - val_loss: 1.2311 - val_accuracy: 0.5000 - val_f1_m: 0.5036 - val_precision_m: 0.5083 - val_recall_m: 0.5000
Epoch 3/20
42/42 [==============================] - 13s 301ms/step - loss: 0.4578 - accuracy: 0.8470 - f1_m: 0.8454 - precision_m: 0.8521 - recall_m: 0.8404 - val_loss: 0.6635 - val_accuracy: 0.6500 - val_f1_m: 0.6583 - val_precision_m: 0.7083 - val_recall_m: 0.6250
Epoch 4/20
42/42 [==============================] - 13s 303ms/step - loss: 0.3886 - accuracy: 0.8352 - f1_m: 0.8581 - precision_m: 0.9005 - recall_m: 0.8264 - val_loss: 0.7776 - val_accuracy: 0.7500 

In [35]:
results = model.evaluate(
    validation_generator,
    steps=val_steps // batch_size,
)

print("loss: " + str(results[0]))
print("acc: " + str(results[1]))
print("f1: " + str(results[2]))
print("pre: " + str(results[3]))
print("rec: " + str(results[4]))

10/10 [==============================] - 3s 309ms/step - loss: 0.4357 - accuracy: 0.8000 - f1_m: 0.8214 - precision_m: 0.8500 - recall_m: 0.8000
loss: 0.4357386529445648
acc: 0.800000011920929
f1: 0.8214284777641296
pre: 0.8500000238418579
rec: 0.800000011920929
